# Using Feature Layers

The feature layer is the primary concept for working with features in a GIS.

Users create, import, export, analyze, edit, and visualize features, i.e. “entities in space” as feature layers.

Feature layers can be added to and visualized using maps. They act as inputs to and outputs from feature analysis tools.

Feature layers are created by publishing feature data to a GIS, and are exposed as a broader resource (Item) in the GIS. Feature layer instances can be obtained through the layers attribute on feature layer collection Items in the GIS. A feature layer collection is a collection of feature layers and tables, with the associated relationships among the entities. A feature layer collection is backed by a [feature service](http://server.arcgis.com/en/server/latest/publish-services/windows/what-is-a-feature-service-.htm) in a web GIS.

## Accessing Feature Layers

Feature layer collection items are available as content in the GIS. You can search the GIS for feature layer collection items, or get them using their item id. 

Feature layers are available through the layers attribute on feature layer collection Items in the GIS.

## Searching the GIS for feature layers 

You can search the GIS for feature layer collections by specifying the item type as 'Feature Layer Collection' or 'Feature Layer'.

Note: A feature layer collection can be considered a type of feature layer such as a group feature layer. Hence, you can specify the item type as 'Feature Layer' and still get back feature layer collection items as results.

The examples below will clarify this further:

In [3]:
# Establish a connection to your GIS.
from arcgis.gis import GIS
from IPython.display import display
gis = GIS() # anonymous connection to www.arcgis.com

In [5]:
# Search for 'USA major cities' feature layer collection
search_results = gis.content.search('title: USA Major Cities',
                                    'Feature Layer')

# Access the first Item that's returned
major_cities_item = search_results[0]

major_cities_item

<Item title:"USA Major Cities" type:Feature Layer Collection owner:esri_dm>

Note that the major_cities_item is a 'Feature Layer Collection' item.
Since this item is a Feature Layer Collection, accessing the `layers` property will give us a list of `FeatureLayer` objects.

In [6]:
major_cities_layers = major_cities_item.layers
major_cities_layers

[<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Major_Cities/FeatureServer/0">]

## Accessing feature layers using item id

Feature layer collection items are available as content in the GIS. You can get them using their item id, and query their layers property to get to the feature layers:

In [7]:
freeways = gis.content.get('91c6a5f6410b4991ab0db1d7c26daacb')
freeways

<Item title:"USA Freeway System" type:Feature Layer Collection owner:esri>

Since freeways is a Feature Layer Collection item, accessing the layers property will give us a list of FeatureLayer objects. This item has two layers:

In [8]:
freeways.layers 

[<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Freeway_System/FeatureServer/1">,
 <FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Freeway_System/FeatureServer/2">]

The code below cycles through the layers and prints their names:

In [9]:
for lyr in freeways.layers:
    print(lyr.properties.name)

USA Freeway System (over 1:500k)
USA Freeway System (below 1:500k)


## Accessing feature layers and tables from feature services

A feature service serves a collection of feature layers and tables, with the associated relationships among the entities. It is represented by `arcgis.features.FeatureLayerCollection` in the ArcGIS Python API.

Instances of FeatureLayerCollection can be constructed using a feature service url, as shown below:

In [10]:
from arcgis.features import FeatureLayerCollection

In [11]:
fs_url = 'http://sampleserver3.arcgisonline.com/ArcGIS/rest/services/SanFrancisco/311Incidents/FeatureServer'
sanfran = FeatureLayerCollection(fs_url)

The collection of layers and tables in a FeatureLayerCollection can be accessed using the layers and tables properties respectively:

In [12]:
sanfran.layers

[<FeatureLayer url:"http://sampleserver3.arcgisonline.com/ArcGIS/rest/services/SanFrancisco/311Incidents/FeatureServer/0">]

**Tables** represent entity classes with uniform properties. In addition to working with “entities with location” as features, the GIS can also work with non-spatial entities as rows in tables. Working with tables is similar to working with feature layers, except that the rows (Features) in a table do not have a geometry, and tables ignore any geometry related operation. 

The sanfran feature layer collection also has a table that can be obtained using its tables property:

In [13]:
sanfran.tables

[<Table url:"http://sampleserver3.arcgisonline.com/ArcGIS/rest/services/SanFrancisco/311Incidents/FeatureServer/1">]

## Accessing feature layers from a feature layer url

Instances of FeatureLayers can also be constructed using a url to the REST endpoint of a feature layer:

In [14]:
from arcgis.features import FeatureLayer

In [15]:
lyr_url = 'http://sampleserver3.arcgisonline.com/ArcGIS/rest/services/SanFrancisco/311Incidents/FeatureServer/0'

layer = FeatureLayer(lyr_url)
layer

<FeatureLayer url:"http://sampleserver3.arcgisonline.com/ArcGIS/rest/services/SanFrancisco/311Incidents/FeatureServer/0">

<a id="featurelayer-properties"></a>
## Properties of FeatureLayer
In this section, let us take a closer look at the properties of a `FeatureLayer` object. We will use the `major_cities_layers` object created earlier

In [16]:
feature_layer = major_cities_layers[0]
feature_layer

<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Major_Cities/FeatureServer/0">

The `properties` field on a `FeatureLayer` object provides a dictionary representation of all its properties. However you can access individual properties as fields as well:

In [17]:
feature_layer.properties.extent

{
  "xmin": -17608123.3895845,
  "ymin": 2237818.89912024,
  "xmax": -7656889.54512499,
  "ymax": 9568526.08684907,
  "spatialReference": {
    "wkid": 102100,
    "latestWkid": 3857
  }
}

The `capabilities` property is useful to know what kinds of edits and operations be performed on the feature layer

In [18]:
feature_layer.properties.capabilities

'Query'

You can access the rendering information from the `drawingInfo` property

In [19]:
feature_layer.properties.drawingInfo.renderer.type

'classBreaks'

<a id="querying-layers"></a>
## Querying feature layers
Querying is a powerful operation that can be performed on a `FeatureLayer` object. Let is take a closer look here. To write meaningful queries, we need to know the names of fields present in the layer. This can be determined by calling the `fields` property:

In [20]:
for f in feature_layer.properties.fields:
    print(f['name'])

FID
OBJECTID
NAME
CLASS
ST
STFIPS
PLACEFIPS
CAPITAL
POP_CLASS
POPULATION
POP2010
WHITE
BLACK
AMERI_ES
ASIAN
HAWN_PI
HISPANIC
OTHER
MULT_RACE
MALES
FEMALES
AGE_UNDER5
AGE_5_9
AGE_10_14
AGE_15_19
AGE_20_24
AGE_25_34
AGE_35_44
AGE_45_54
AGE_55_64
AGE_65_74
AGE_75_84
AGE_85_UP
MED_AGE
MED_AGE_M
MED_AGE_F
HOUSEHOLDS
AVE_HH_SZ
HSEHLD_1_M
HSEHLD_1_F
MARHH_CHD
MARHH_NO_C
MHH_CHILD
FHH_CHILD
FAMILIES
AVE_FAM_SZ
HSE_UNITS
VACANT
OWNER_OCC
RENTER_OCC


The query method has a number of parameters that allow you to refine and transform the results. Since the processing is performed on the server, this operation is not restricted by the capacity of the client computer. 

For instance, let us select all the cities whose population in the year 2007 was greater than 1 million. Instead of returning all the fields, let us get only population related fields

In [23]:
query_result1 = feature_layer.query(where='POP2010>1000000', 
                                    out_fields='WHITE,BLACK,MULT_RACE,HISPANIC')
len(query_result1.features)

9

In [24]:
query_result1.fields

[{'name': 'FID',
  'type': 'esriFieldTypeOID',
  'alias': 'FID',
  'sqlType': 'sqlTypeInteger',
  'domain': None,
  'defaultValue': None},
 {'name': 'WHITE',
  'type': 'esriFieldTypeInteger',
  'alias': 'WHITE',
  'sqlType': 'sqlTypeInteger',
  'domain': None,
  'defaultValue': None},
 {'name': 'BLACK',
  'type': 'esriFieldTypeInteger',
  'alias': 'BLACK',
  'sqlType': 'sqlTypeInteger',
  'domain': None,
  'defaultValue': None},
 {'name': 'MULT_RACE',
  'type': 'esriFieldTypeInteger',
  'alias': 'MULT_RACE',
  'sqlType': 'sqlTypeInteger',
  'domain': None,
  'defaultValue': None},
 {'name': 'HISPANIC',
  'type': 'esriFieldTypeInteger',
  'alias': 'HISPANIC',
  'sqlType': 'sqlTypeInteger',
  'domain': None,
  'defaultValue': None}]

If we are only interested in the count, we could save bandwidth by setting the `return_count_only` to `True`

In [25]:
feature_layer.query(where='POP2010>1000000', return_count_only=True)

9

## Querying features using a different spatial reference

In [26]:
query_result1.spatial_reference

{'wkid': 102100, 'latestWkid': 3857}

By default, the query results are in the same spatial reference as the source layer. However you can use the `out_sr` parameter to reproject the result into a desired spatial reference. The projection happens on the server and on all the resulting features.

In the example above, we obtained data in wkid:3857, a well known id for 'Web Mercator' projection. We can observe how the coordinates look like below:

In [27]:
query_result1.features[0].geometry

{'x': -13195807.345545301, 'y': 4052488.8059764393}

The coordinates are in projected coordinate system as expected. If we wish to have this data in latitude and longitude instead, we could do so by changing the `out_sr` to wkid:4326

<a id = "featureset-properties"></a>
## FeatureSet properties
As seen previously, a `FeatureSet` is returned by a `query()` operation. The `FeatureSet` object packs a bunch of useful properties that help us discern useful information about the features under access

One of the important properties is the `spatial_reference` as you saw earlier. Below, we are using the same `query_result1` `FeatureSet` from earlier query operation.

In [28]:
query_result1.spatial_reference

{'wkid': 102100, 'latestWkid': 3857}

One of the most powerful operation on a `FeatureSet` is accessing the features not as `Feature` objects, but as pandas dataframe objects. The `df` property, returns a dataframe object:

In [29]:
query2 = feature_layer.query(where="POP2010 > 1000000")
query2.sdf

,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_5_9,AGE_65_74,AGE_75_84,...,PLACEFIPS,POP2010,POPULATION,POP_CLASS,RENTER_OCC,SHAPE,ST,STFIPS,VACANT,WHITE
0,237462,274373,314543,638900,570467,503164,374391,231528,209059,128659,...,0644000,3792621,3887115,10,814305,"{""x"": -13195807.345545301, ""y"": 4052488.805976...",CA,06,95827,1888158
1,75345,90813,127987,230247,183750,171993,131765,75073,72013,46643,...,0666000,1307402,1343525,10,249934,"{""x"": -13038647.444455143, ""y"": 3877209.499119...",CA,06,32941,769971
2,78234,76976,94880,219848,170106,150317,108830,89707,57380,33881,...,4819000,1197816,1254907,10,256177,"{""x"": -10771976.460229648, ""y"": 3868099.451091...",TX,48,58582,607415
3,137307,142544,171086,373985,291218,268661,202641,151041,106310,60257,...,4835000,2099451,2216413,10,427407,"{""x"": -10616276.96221088, ""y"": 3477489.7999848...",TX,48,110003,1060491
4,97421,102486,107623,197161,175669,174799,133845,99319,74611,45911,...,4865000,1327407,1380401,10,208572,"{""x"": -10966982.38876809, ""y"": 3424119.2597500...",TX,48,44604,963413
5,164466,182933,223027,515551,378139,338737,262849,166077,151095,89804,...,1714000,2695598,2737877,10,575998,"{""x"": -9761403.70804745, ""y"": 5136934.75558690...",IL,17,148777,1212835
6,108732,110352,106757,224128,206387,195144,139018,113260,70456,36981,...,0455000,1445632,1513274,10,218064,"{""x"": -12477855.755940389, ""y"": 3971403.536437...",AZ,04,75343,951958
7,468154,535833,642585,1392445,1154687,1107376,890012,473159,531461,320291,...,3651000,8175133,8433086,10,2146892,"{""x"": -8223399.012453097, ""y"": 4966718.4366832...",NY,36,261278,3597341
8,90640,118297,146717,246062,188323,197970,160808,90827,94764,62434,...,4260000,1526006,1551773,10,275200,"{""x"": -8363869.269200069, ""y"": 4867065.3309799...",PA,42,70435,626221


Accessing the features as a dataframe makes if easier to analyze the data statistically.

## Accessing Features from query results

In [30]:
query_geographic = feature_layer.query(where='POP2010 > 1000000', out_sr='4326')
query_geographic.features[0].geometry

{'x': -118.53995424799996, 'y': 34.17622092200002}

We can execute the `query()` method on the first `FeatureLayer` object and get a `FeatureSet`. Let us query and access the first 10 features in this layer

In [31]:
major_cities_l1 = major_cities_layers[0]
major_cities_l1_fset = major_cities_l1.query(where= 'FID < 11')
type(major_cities_l1_fset)

arcgis.features.feature.FeatureSet

Now, accessing the `features` property of the above `FeatureSet` object will provide us the individual point `Features`.

In [32]:
major_cities_l1_features = major_cities_l1_fset.features
len(major_cities_l1_features)

10

<a id="feature-properties"></a>
## Accessing Feature geometry and attributes
As mentioned earlier, the `Feature` object is a fine grained representation of spatial information. Two important properties of a `Feature` object are its `geometry` and `attributes`:

Let us display the geometry and attributes of the first feature

In [33]:
major_cities_l1_features[0].geometry

{'x': -12751215.004681978, 'y': 4180278.406256967}

In [34]:
major_cities_l1_features[0].attributes

{'FID': 1,
 'OBJECTID': 1,
 'NAME': 'Bullhead City',
 'CLASS': 'city',
 'ST': 'AZ',
 'STFIPS': '04',
 'PLACEFIPS': '0408220',
 'CAPITAL': ' ',
 'POP_CLASS': 6,
 'POPULATION': 40346,
 'POP2010': 39540,
 'WHITE': 32367,
 'BLACK': 508,
 'AMERI_ES': 450,
 'ASIAN': 556,
 'HAWN_PI': 59,
 'HISPANIC': 9386,
 'OTHER': 4426,
 'MULT_RACE': 1174,
 'MALES': 19535,
 'FEMALES': 20005,
 'AGE_UNDER5': 2142,
 'AGE_5_9': 2067,
 'AGE_10_14': 2144,
 'AGE_15_19': 2314,
 'AGE_20_24': 2002,
 'AGE_25_34': 3531,
 'AGE_35_44': 3887,
 'AGE_45_54': 5643,
 'AGE_55_64': 6353,
 'AGE_65_74': 5799,
 'AGE_75_84': 2850,
 'AGE_85_UP': 808,
 'MED_AGE': 48.2,
 'MED_AGE_M': 47.7,
 'MED_AGE_F': 48.6,
 'HOUSEHOLDS': 16761,
 'AVE_HH_SZ': 2.35,
 'HSEHLD_1_M': 3141,
 'HSEHLD_1_F': 3013,
 'MARHH_CHD': 1988,
 'MARHH_NO_C': 5484,
 'MHH_CHILD': 614,
 'FHH_CHILD': 1301,
 'FAMILIES': 10512,
 'AVE_FAM_SZ': 2.82,
 'HSE_UNITS': 23464,
 'VACANT': 6703,
 'OWNER_OCC': 10198,
 'RENTER_OCC': 6563}

<a id="features-from-a-feature-collection"></a>
# Using feature collections

Similar to feature layers, [feature collections](http://doc.arcgis.com/en/arcgis-online/reference/feature-collections.htm) can also be used to store features. With a feature collection, a service is not created to serve out feature data. Instead it is stored as json data with the item. Feature collections can be added to maps as layers, passed as input to feature analysis tools and queried for feature data.

Feature collections are shared in the GIS as items. Feature Collection Items can be searched by specifying 'Feature Collection' as the item_type.

Let us search for a feature collection published by Iowa Dept. of Transportation as an example:

In [35]:
search_fc = gis.content.search("title:AVL_Direct_FC", item_type='Feature Collection')
iowa_fc_item = search_fc[0]
iowa_fc_item

<Item title:"AVL_Direct_FC" type:Feature Collection owner:IowaDOT_SODA>

Accessing the `layers` property on a feature collection item returns a list of `FeatureCollection` objects

In [36]:
iowa_fc_item.layers

[<FeatureCollection>]

In [37]:
iowa_fc = iowa_fc_item.layers[0]

You can call the `query()` method on a `FeatureCollection` object to get a `FeatureSet`.

In [38]:
iowa_fset = iowa_fc.query()

Once you have a `FeatureSet` object, you can access the `features` property to get a list of `Feature` objects as seen earlier

In [39]:
iowa_features = iowa_fset.features
iowa_features[0].geometry

{'x': -10617585.164750207,
 'y': 5160968.961654072,
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}